In [1]:
import time
import pickle
import holidays
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt

from tqdm import tqdm
from datetime import datetime
from selenium import webdriver
from collections import defaultdict
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from selenium.webdriver.support import expected_conditions as EC

# 삼성전자 뉴스 크롤링
- 기간 : 2021년 1월 1일 ~ 현재
- 경제, 국제, IT_과학 카테고리
- 매일경제, 서울경제, 아주경제, 한국경제 : 주요 한국 경제 언론사

In [354]:
driver = webdriver.Chrome()
driver.get("https://www.bigkinds.or.kr/v2/news/index.do")

In [353]:
driver.close()

## 1. 뉴스 필터링

#### 1-1. 언론사 필터링

In [ ]:
# 언론사 선택

# 매일경제, 서울경제, 아주경제, 한국경제 클릭
# find_media = ['매일경제', '서울경제', '아주경제', '한국경제']

# 전국일간지
media1 = driver.find_element(By.CSS_SELECTOR, "#category_provider_group > li:nth-child(1) > a")
media1.click()

# 경제일간지
media2 = driver.find_element(By.CSS_SELECTOR, "#category_provider_group > li:nth-child(2) > a > label")
media2.click()

# 방송사
media3 = driver.find_element(By.CSS_SELECTOR, "#category_provider_group > li:nth-child(5) > a > label")
media3.click()

# 전문지
media4 = driver.find_element(By.CSS_SELECTOR, "#category_provider_group > li:nth-child(6) > a > label")
media4.click()

# 인터넷 신문
media5 = driver.find_element(By.CSS_SELECTOR, "#category_provider_group > li:nth-child(8) > a > label")
media5.click()

#### 1-2. 검색 기간 설정

In [356]:
date_filter = "2025-05-01"
date_end = "2025-05-20"

# 기간 설정
duration_news = driver.find_element(By.CSS_SELECTOR, "#collapse-step-1-body > div.srch-detail.v2 > div > div.tab-btn-wp1 > div.tab-btn-inner.tab1 > a")
duration_news.click()

time.sleep(0.5)

# 기간 직접 입력
start_date = driver.find_element(By.CSS_SELECTOR, "#search-begin-date")
start_date.send_keys(Keys.COMMAND + "A")
time.sleep(1)
start_date.send_keys(Keys.DELETE)
time.sleep(0.2)
start_date.send_keys(date_filter)
time.sleep(1)

end_date = driver.find_element(By.CSS_SELECTOR, "#search-end-date")
end_date.send_keys(Keys.COMMAND + "A")
time.sleep(1)
end_date.send_keys(Keys.DELETE)
time.sleep(0.2)
end_date.send_keys(date_end)
time.sleep(0.5)
end_date.send_keys(Keys.ENTER)

#### 1-3. 카테고리 필터링

In [ ]:
# 통합 분류 : 경제,  국제, IT_과학
general_category = driver.find_element(By.CSS_SELECTOR, "#collapse-step-1-body > div.srch-detail.v2 > div > div.tab-btn-wp2 > div.tab-btn-inner.tab3 > a")
general_category.click()

# 경제
economy_checkbox = driver.find_element(By.CSS_SELECTOR, "#srch-tab3 > ul > li:nth-child(2) > div > span:nth-child(3)")
economy_checkbox.click()

# 국제
world_checkbox = driver.find_element(By.CSS_SELECTOR, "#srch-tab3 > ul > li:nth-child(5) > div > span:nth-child(3)")
world_checkbox.click()

# IT_과학
it_checkbox = driver.find_element(By.CSS_SELECTOR, "#srch-tab3 > ul > li:nth-child(8) > div > span:nth-child(3)")
it_checkbox.click()

#### 1-4. 상세 조건 설정
- 제목 : "삼성전자" 필수 포함
- "[속보]", "[스팟]", "칼럼" 제외

In [ ]:
## 상세검색 조건 설정
detail_search = driver.find_element(By.CSS_SELECTOR, "#collapse-step-1-body > div.srch-detail.v2 > div > div.tab-btn-wp3 > div.tab-btn-inner.tab5 > a")
detail_search.click()

# 검색어 범위 설정
search_range = driver.find_element(By.CSS_SELECTOR, "#search-scope-type")
search_range.click()

# 제목 검색 설정
filter_title = driver.find_element(By.CSS_SELECTOR, "#search-scope-type > option:nth-child(2)")
filter_title.click()

# 단어 중 1개 이상 포함 : "삼성전자"
filter_samsung = driver.find_element(By.CSS_SELECTOR, "#orKeyword1")
filter_samsung.send_keys("삼성전자")

# 제외 단어 설정
filter_except = driver.find_element(By.CSS_SELECTOR, "#notKeyword1")
filter_except.send_keys("[속보] OR [스팟] OR 칼럼")

#### 1-5. 기타 설정

In [358]:
# 최종 검색 버튼 클릭
search_btn = driver.find_element(By.CSS_SELECTOR, "#detailSrch1 > div.srch-foot > div > button.btn.btn-md.btn-primary.news-search-btn")
search_btn.click()

## 로딩 화면 ##

In [359]:
# 분석 기사 클릭
# label 요소를 직접 클릭 (for="filter-tm-use"로 연결되어 있음)
label = driver.find_element(By.CSS_SELECTOR, 'label[for="filter-tm-use"]')

# JS로 강제 클릭 (보통 이게 가장 확실함)
driver.execute_script("arguments[0].click();", label)

# time.sleep(1)

## 로딩 화면 ##

In [360]:
# 보기 정렬 : 과거순
sort_articles = driver.find_element(By.CSS_SELECTOR, "#select1")
sort_articles.click()

sort_ascending = driver.find_element(By.CSS_SELECTOR, "#select1 > option:nth-child(3)")
sort_ascending.click()

## 로딩 화면 ##

In [361]:
# 보기 개수 : 100개
view_articles = driver.find_element(By.CSS_SELECTOR, "#select2")
view_articles.click()

view_hunnit = driver.find_element(By.CSS_SELECTOR, "#select2 > option:nth-child(4)")
view_hunnit.click()

## 로딩 화면 ##

## 2. 뉴스 기사 크롤링
- 날짜
- 제목
- 본문 요약

In [362]:
# 전체 보기 페이지 개수
total_page = int(driver.find_element(By.CSS_SELECTOR, "#news-results-tab > div.data-result-btm.m-only.paging-v3-wrp > div.btm-pav-wrp > div > div > div > div:nth-child(6) > div").text)

total_page

32

In [ ]:
# 뉴스 크롤링
news_by_date = defaultdict(list)
print(f"==========전체 페이지 수 : {total_page}==========")

for i in range(1, total_page+1) :
    print(f"현재까지 수집된 기사 개수 : {len(news_by_date)}")
    print(f"=========={i} 페이지 뉴스 기사 크롤링 시작==========")
    
    article_cnt = int(driver.find_element(By.CSS_SELECTOR, "#news-results-tab > div.data-result-hd.m-only > h3 > span.total-news-cnt").text)
                                                            #news-results-tab > div.data-result-hd.m-only > h3 > span.total-news-cnt
                                                            #news-results-tab > div.data-result-hd.pc-only.paging-v2-wrp > h3 > span.total-news-cn
    if article_cnt == 0 :
        print(f"{date_filter} 날짜의 기사는 없습니다.")
        date_obj = datetime.strptime(date_filter, '%Y-%m-%d')
        news_by_date[date_obj] = []
    else :
        # 한 페이지 전체 뉴스 기사 리스트
        article_list = driver.find_elements(By.CSS_SELECTOR, "#news-results > div")

    for j in range(1, len(article_list)+1) :
        tmp_article = driver.find_element(By.CSS_SELECTOR, f"#news-results > div:nth-child({j}) > div > div.cont > a")

        # 기사 제목
        title = tmp_article.find_element(By.TAG_NAME, "span").text
        # 본문
        summary_element = tmp_article.find_element(By.TAG_NAME, "p")
        summary_html = summary_element.get_attribute("innerHTML")

        # <br>를 줄바꿈 문자로 변환
        parts = summary_html.replace('<br>', '\n').replace('<br/>', '\n').split('\n')

        # 각 줄 공백 정리
        parts = [part.strip() for part in parts if part.strip()]

        # 마지막 줄 점검
        if parts and '..' in parts[-1]:
            parts = parts[:-1]  # 마지막 문장이 ".." 포함이면 버림

        # 최종 텍스트 생성
        summary_text = ' '.join(parts)

        # 제목 + 구분자 + 본문
        SEPERATOR = " ||| "
        final_article = title + SEPERATOR + summary_text

        # 날짜
        dt = driver.find_element(By.CSS_SELECTOR, f"#news-results > div:nth-child({j}) > div > div.cont > div > p:nth-child(2)").text
        # datetime 객체로 변환
        date = datetime.strptime(dt, '%Y/%m/%d')

        # 원하는 형식의 문자열로 변환
        article_date = date.strftime('%Y-%m-%d')

        # 딕셔너리 저장
        news_by_date[date].append(final_article)

    # 현재 페이지
    current_input = driver.find_element(By.CSS_SELECTOR, "#paging_news_result")
    current_page = int(current_input.get_attribute("value"))

    if current_page == total_page :
        print("마지막 페이지에 도달했습니다.")
        break

    # 다음 페이지 : 입력창에 페이지 번호 직접 입력
    next_button = driver.find_element(By.CSS_SELECTOR, "#news-results-tab > div.data-result-btm.m-only.paging-v3-wrp > div.btm-pav-wrp > div > div > div > div:nth-child(7) > a")
    driver.execute_script("arguments[0].click();", next_button)

    WebDriverWait(driver, 200).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, "#collapse-step-2-body > div > div.data-result.loading-cont > div.news-loader.loading > div")))

    new_input = driver.find_element(By.CSS_SELECTOR, "#paging_news_result")
    new_page = int(new_input.get_attribute("value"))

==========전체 페이지 수 : 32==========
현재까지 수집된 기사 개수 : 0
==========1 페이지 뉴스 기사 크롤링 시작==========
현재까지 수집된 기사 개수 : 1
==========2 페이지 뉴스 기사 크롤링 시작==========
현재까지 수집된 기사 개수 : 2
==========3 페이지 뉴스 기사 크롤링 시작==========
현재까지 수집된 기사 개수 : 2
==========4 페이지 뉴스 기사 크롤링 시작==========
현재까지 수집된 기사 개수 : 4
==========5 페이지 뉴스 기사 크롤링 시작==========
현재까지 수집된 기사 개수 : 6
==========6 페이지 뉴스 기사 크롤링 시작==========
현재까지 수집된 기사 개수 : 7
==========7 페이지 뉴스 기사 크롤링 시작==========
현재까지 수집된 기사 개수 : 7
==========8 페이지 뉴스 기사 크롤링 시작==========
현재까지 수집된 기사 개수 : 8
==========9 페이지 뉴스 기사 크롤링 시작==========
현재까지 수집된 기사 개수 : 8
==========10 페이지 뉴스 기사 크롤링 시작==========
현재까지 수집된 기사 개수 : 8
==========11 페이지 뉴스 기사 크롤링 시작==========
현재까지 수집된 기사 개수 : 9
==========12 페이지 뉴스 기사 크롤링 시작==========
현재까지 수집된 기사 개수 : 9
==========13 페이지 뉴스 기사 크롤링 시작==========
현재까지 수집된 기사 개수 : 11
==========14 페이지 뉴스 기사 크롤링 시작==========
현재까지 수집된 기사 개수 : 12
==========15 페이지 뉴스 기사 크롤링 시작==========
현재까지 수집된 기사 개수 : 12
==========16 페이지 뉴스 기사 크롤링 시작==========
현재까지 수집된 기사 개수 : 13
========

### 데이터프레임 저장

In [364]:
date_name = (date_filter.split('-')[0][2:] + date_filter.split('-')[1] + date_filter.split('-')[2])[:4]

date_name

'2505'

In [ ]:
# 최대 기사 수를 가진 데이터의 컬럼 개수 지정
max_articles = max((len(articles) for articles in news_by_date.values()), default=0)
print(max_articles)

# 최대 기사 수를 컬럼수로 지정
columns = ['date'] + [str(i) for i in range(1, max_articles+1)] + ['news_count']

data_rows = []
for date, articles in news_by_date.items() :
    date_str = date.strftime('%Y-%m-%d')
    news_count = len(articles)

    row = [date_str] + articles
    # 최대 기사 수보다 적은 기사 수를 가진 날에는 빈 컬럼값에 NaN 처리
    if len(articles) < max_articles :
        row += [pd.NA] * (max_articles - len(articles))

    row += [news_count]
    data_rows.append(row)

# 데이터프레임 생성
df = pd.DataFrame(data_rows, columns=columns)
df.to_csv(f'/Users/taeheon/stock_price/data/{date_name}_headlines.csv')

345


In [ ]:
driver.close()

# * missing date 확인
- 누락된 날짜가 있는지 확인

In [105]:
# 기준 날짜 범위
full_date_range = pd.date_range(start="2021-01-01", end="2025-04-28")

# 누락된 날짜 찾기
missing_dates = full_date_range.difference(df['date'])

# 한국 공휴일 객체 생성
kr_holidays = holidays.KR(years=range(2021, 2026))

# 전체 달력용 DataFrame 생성
calendar_df = pd.DataFrame({'date': full_date_range})
calendar_df['weekday'] = calendar_df['date'].dt.weekday  # 0=월요일, 6=일요일
calendar_df['is_weekend'] = calendar_df['weekday'] >= 5
calendar_df['is_holiday'] = calendar_df['date'].isin(kr_holidays)
calendar_df['is_weekend_or_holiday'] = calendar_df['is_weekend'] | calendar_df['is_holiday']

# 집계
total_days = len(full_date_range)
weekend_days = calendar_df['is_weekend'].sum()
holiday_days = calendar_df['is_holiday'].sum()
weekend_or_holiday_days = calendar_df['is_weekend_or_holiday'].sum()

# 출력
print("\n--- 전체 기간 내 휴일 집계 ---")
print(f"전체 날짜 수: {total_days}일")
print(f"주말 일수: {weekend_days}일")
print(f"공휴일 일수: {holiday_days}일")
print(f"주말 또는 공휴일(중복 제거): {weekend_or_holiday_days}일")

# df의 복사본 생성 및 주말/공휴일 여부 계산
df_dates = pd.DataFrame({'date': pd.to_datetime(df['date'])})
df_dates['weekday'] = df_dates['date'].dt.weekday
df_dates['is_weekend'] = df_dates['weekday'] >= 5
df_dates['is_holiday'] = df_dates['date'].isin(kr_holidays)
df_dates['is_weekend_or_holiday'] = df_dates['is_weekend'] | df_dates['is_holiday']

# 주말/공휴일이면서 데이터가 존재하는 날짜 추출
existing_on_weekend_or_holiday = df_dates[df_dates['is_weekend_or_holiday']]

# 출력
print("\n--- 주말 또는 공휴일이지만 df에 존재하는 날짜 ---")
print(f"총 {len(existing_on_weekend_or_holiday)}일")
print(existing_on_weekend_or_holiday[['date', 'is_weekend', 'is_holiday']])

# 누락된 날짜가 주말 or 공휴일인지 확인
result = pd.DataFrame({'date': missing_dates})
result['weekday'] = result['date'].dt.day_name()
result['is_weekend'] = result['date'].dt.weekday >= 5  # 토요일(5), 일요일(6)
result['is_holiday'] = result['date'].isin(kr_holidays)

# 주말도 아니고 공휴일도 아닌 날짜만 필터링 (정말 빠진 날짜)
truly_missing = result[~(result['is_weekend'] | result['is_holiday'])]

# 출력
print("총 누락 날짜 수:", len(missing_dates))
print("주말 혹은 공휴일 제외하고 실제 누락된 날짜 수:", len(truly_missing))
print(truly_missing)
print(type(truly_missing))


--- 전체 기간 내 휴일 집계 ---
전체 날짜 수: 1579일
주말 일수: 452일
공휴일 일수: 80일
주말 또는 공휴일(중복 제거): 511일

--- 주말 또는 공휴일이지만 df에 존재하는 날짜 ---
총 417일
           date  is_weekend  is_holiday
0    2021-01-01       False        True
1    2021-01-03        True       False
7    2021-01-09        True       False
8    2021-01-10        True       False
14   2021-01-17        True       False
...         ...         ...         ...
1460 2025-04-13        True       False
1466 2025-04-19        True       False
1467 2025-04-20        True       False
1473 2025-04-26        True       False
1474 2025-04-27        True       False

[417 rows x 3 columns]
총 누락 날짜 수: 103
주말 혹은 공휴일 제외하고 실제 누락된 날짜 수: 9
         date    weekday  is_weekend  is_holiday
3  2021-02-04   Thursday       False       False
10 2021-11-04   Thursday       False       False
21 2022-08-11   Thursday       False       False
22 2022-08-12     Friday       False       False
25 2022-09-06    Tuesday       False       False
51 2023-10-20     Friday       

### 누락된 날도 데이터프레임에 채우기

In [ ]:
# 기준 날짜 범위
full_date_range = pd.date_range(start="2021-01-01", end="2025-04-28")

completed_df = df.copy()

completed_df['date'] = pd.to_datetime(completed_df['date'])  # 기존 df의 date 컬럼을 datetime으로 변환
missing_dates = full_date_range.difference(completed_df['date'])

# 3. 누락된 날짜를 datetime으로 명시적 변환
missing_df = pd.DataFrame({'date': pd.to_datetime(missing_dates)})

# 4. 병합 후 정렬
completed_df = pd.concat([completed_df, missing_df], ignore_index=True)
completed_df = completed_df.sort_values('date').reset_index(drop=True)

# 5. 누락 여부 표시 (선택)
completed_df['is_missing'] = completed_df['date'].isin(missing_dates)

# 6. 출력
print("추가된 누락 날짜 수:", len(missing_df))
print("누락 추가 전 vs 누락 추가 후 : ", len(df), "->", len(completed_df))

추가된 누락 날짜 수: 103
누락 추가 전 vs 누락 추가 후 :  1476 -> 1579


### 날짜별 뉴스 개수 컬럼 생성
- news_count

In [ ]:
# date 제외 컬럼만 확인
article_columns = [col for col in concat_df.columns if col.isdigit()]

# NaN이 아닌 셀의 수 = 실제 기사 개수
concat_df['news_count'] = concat_df[article_columns].notna().sum(axis=1)

concat_df.head(5)

,date,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,is_missing,news_count
0,2021-01-01,"아이폰12, 출시 두 달만에 삼성전자 1년치 5G폰 판매량 앞질러 ||| 아이폰12...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,False,1
1,2021-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,0
2,2021-01-03,"""연말정산, 삼성 패스에서 하세요"" 삼성전자도 민간 인증서 전쟁에 나섰다 ||| 삼...","삼성전자, 비스포크 인덕션 신제품 출시…3300W ‘최고 화력’ ||| 삼성전자가 ...",코스피 시총의 25%는 삼성전자 ||| 삼성전자(005930)가 코스피 전체 시가총...,"삼성전자, ‘국내 최대 화력’ 비스포크 인덕션 출시 ||| 삼성전자, ‘국내 최대 ...","""삼성전자 여전히 저평가""…전문가 74%, 새해 톱픽 추천 |||",<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,False,5
3,2021-01-04,"삼성전자, 5G를 넘어 6G까지 선점…시스템반도체 133조 공격적 투자 ||| ◆ ...","삼성전자, 스웨덴 통신업체 에릭슨에 특허소송 추가 피소 ||| 삼성전자가 스웨덴 통...","삼성전자, '갤럭시 언팩 2021' 초대장 발송... 15일 0시 갤S21 공개 |...",“삼성전자 올 1분기 실적 바닥 지난다...목표가 10만원” ||| 키움증권이 4일...,"한달 빨라진 삼성전자 언팩행사, 14일 `요술봉 든 괴물폰` 나온다 ||| 이미 예...",‘삼성전자 시무식’ 김기남 부회장 “100년 사업 기틀 마련의 원년으로 삼자” ||...,[2021 신년사] 김기남 삼성전자 부회장 “업계 판도 주도해 나가자” ||| 김기...,"""2021년 미니 LED TV시대…세진티에스, 삼성전자 핵심 공급업체로 주목"" ||...","김기남 삼성전자 부회장 신년사 ""2021년은 변화 대응하고 미래 준비하는 원년"" |...",...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,False,19
4,2021-01-05,"삼성전자, SAMSUNG이 그리는 AI 빅피처 ||| ◆ 2021 신년기획 미래산업 ◆","삼성전자, `갤럭시 캠퍼스 스토어` 오픈…대학·대학원생 특별 혜택 ||| 삼성전자가...","자고나면 높아지는 삼성전자 목표가... ""11만원으로 상향"" ||| 하나금융투자는 ...","삼성물산 패션, '삼성전자 세일 페스타' 공동 마케팅 진행 진행 ||| 삼성물산, ...","삼성전자, 최고 목표가 '11만1000원'까지 나왔다…역대 최고 ||| 유가증권시장...","삼성전자, 대학생·대학원생을 위한 ‘갤럭시 캠퍼스 스토어’ 오픈 ||| 삼성전자(0...","삼성전자, AI 탑재 세탁기·건조기 CES 출품…美시장 본격 공략 ||| 삼성전자가...","삼성전자, ‘비스포크’ 냉장고 3월 미국 출시 ||| 삼성전자가 라이프스타일 맞춤형...","삼성전자, '비스포크' 세계 최대 가전 시장 미국 출시 ||| 삼성전자가 라이프스타...",...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,False,14


In [115]:
# fulfill.csv로 저장
completed_df.to_csv('0428_headlines_fulfill.csv')